In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%env SEISMIQB_ANONYMIZE 1

import sys
import numpy as np
from copy import copy

PROJECT_PATH = '../' * 3
sys.path.insert(0, f'{PROJECT_PATH}seismiqb')
sys.path.insert(0, f'{PROJECT_PATH}src')

from seismiqb import SeismicDataset, HorizonMetrics, SeismicSampler

In [ ]:
from glob import glob
GEOMETRY_PATH = '/data/seismic_data/seismic_interpretation/001_YETYPUR/001_YETYPUR.blosc'
HORIZONS_PATH = glob('/data/seismic_data/seismic_interpretation/001_YETYPUR/INPUTS/HORIZONS/RAW/*')[1:4]

dataset = SeismicDataset({GEOMETRY_PATH: {'horizons': HORIZONS_PATH}})
print(dataset)

In [ ]:
field = dataset[0]
horizon = field.horizons[0]

## Field attributes

Use `mode='hist'` to plot histograms for desired field attributes.

In [ ]:
_ = field.show(
    attributes=[
        'snr',
        'horizons:*/depths',
        'horizons:*/amplitudes',
        'horizons:*/instant_amplitudes'
    ],
    mode='hist',
    combine='separate'
)

In [ ]:
field.show(
    attributes=[f'horizons:{i}/depths' for i in range(len(field.horizons))],
    mode='hist',
    ratio=.5,
    title_pattern='Depths distribution of {label_name}',
)

## Metrics

Some functions, like `horizon.compare` already use `mode='hist'` under the hood.

In [ ]:
def gkern(size, sigma):
    x = np.linspace(-(size - 1) / 2., (size - 1) / 2., size)
    gauss = np.exp(-0.5 * np.square(x) / np.square(sigma))
    kernel = np.outer(gauss, gauss)
    return kernel / np.sum(kernel)

def add_horizon_anomalies(horizon):
    shifts = np.zeros(horizon.matrix.shape, dtype=np.int32)

    for _ in range(np.random.randint(10, 20)):
        size = np.random.randint(10, 50)
        kernel = gkern(size, size *.3)
        kernel = kernel / kernel.max() * np.random.randint(5, 10)
        kernel = np.repeat(kernel, np.random.randint(1,4), axis=np.random.randint(2))

        i, x, h = horizon.points[np.random.choice(len(horizon))]
        i_start = i - kernel.shape[0] // 2
        i_stop = i_start + kernel.shape[0]
        x_start = x - kernel.shape[1] // 2
        x_stop = x_start + kernel.shape[1]

        if (i_start < 0) or (x_start < 0) or (i_stop > shifts.shape[0]) or (x_stop > shifts.shape[1]):
            continue

        shifts[i_start : i_stop, x_start : x_stop] += kernel.astype(np.int32)

    shifts[horizon.matrix < 0] = horizon.FILL_VALUE
    shifter = lambda matrix: matrix + shifts

    shifted = copy(horizon)
    shifted.name = f"shifted {horizon.name}"
    shifted.apply_to_matrix(shifter)
    
    return shifted

Note that additional arguments for histogram might be provided with `hist_` prefix.

In [ ]:
anomalous = add_horizon_anomalies(horizon)
_ = horizon.compare(anomalous, printer=lambda _: None, ignore_zeros=True)

One can provide parameters for those plots histograms via `hist_kwargs`.

In [ ]:
_ = horizon.compare(
    anomalous,
    printer=lambda _: None,
    ignore_zeros=True,
    hist_kwargs=
    {
        'color': 'lightcoral',
        'hist_log': True,
        'hist_cumulative': -1,
        'hist_histtype': 'step',
        'hist_linewidth': 3,
        'hist_linestyle': '--',
        'title': 'Cumulative histogram of horizon depths differences'
    }
)